In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

# RK integration for multiple coupled derivatives

In [ ]:
#define our coupled derivatives to integrate
def dydx(x,y):
    
    #y is a 2 element array, with y[0] set to y & y[1] set to z
    #declare (2-element) array for yderivs --> where dydx (deriv of y) is z, and dzdx (deriv of z, second deriv of y) is -y
    #^ our system of equations to evolve
    
    y_derivs = np.zeros(2)
    
    #dydx = z
    y_derivs[0] = y[1] 
    
    #dzdx = -y
    y_derivs[1] = -1*y[0]
    
    return y_derivs


In [ ]:
#define rk multi-variable core
def rk_mv_core(dydx, xi, yi, nv, h):
    #def k arrays for 4^ RK integration
    #nv = both variables z&y at the same time
    
    k1 = np.zeros(nv)
    k2 = np.zeros(nv)
    k3 = np.zeros(nv)
    k4 = np.zeros(nv)
    
    #ipoh = i + 1/2
    #ipo = i +1
    #define x at a half step
    x_ipoh = xi + 0.5*h
    
    #define x at one step
    x_ipo = xi + h
    
    #declare temporary y array, starting with zeros for z&y, later updated values
    y_temp = np.zeros(nv)
    
    #get k1, then k2, then k3, then k4 values (refinement of grid/updated with steps)
    #^ wrt to both y &z
    #[:] = all elements in the array
    #get k1 values
    y_derivs = dydx(xi, yi) 
    #^deriv of y, with xi&yi
    k1[:] = h*y_derivs[:] 
    #^w/ 0 = z & 1 = -y
    
    #get k2 values --> updated from k1 with a half step (half of k1)
    y_temp[:] = yi[:] + 0.5*k1[:] 
    #^yi, for z & y, for updated y_temp
    y_derivs[:] = dydx(x_ipoh, y_temp) 
    #^dydx variables, with x @ 1/2 step for x & updated ytemp for y
    k2[:] = h*y_derivs[:] 
    #^k2 equal to updated yderiv values*h(step)
    
    #k3 - updated from k2 with another half step (half of k2, quarter of k1)
    y_temp[:] = yi[:] + 0.5*k2[:] 
    #^yi, for z & y, for updated y_temp, and half-step refined grid from k2
    y_derivs[:] = dydx(x_ipoh, y_temp) 
    #^dydx variables, with x @ 1/2 step for x & updated ytemp for y
    k3[:] = h*y_derivs[:] 
    #^updated yderivs*h for k3
    
    #k4 - updated from k3 with a step (same as k3, last quarter of k1 to equal a full step h)
    y_temp[:] = yi[:] + k3[:] 
    #^yi, for z & y, for updated y_temp, and a step refined grid from k3
    y_derivs[:] = dydx(x_ipo, y_temp) 
    #^dydx variables, with x @ 1 step for x & updated ytemp for y
    k4[:] = h*y_derivs[:] 
    #^updated ydervis*h for k4 
    
    #advance y by step h
    #^equal to yi + weighted sum of all k derivatives
    
    yipo = yi + (k1+2*k2+2*k3+k4)/6
    return yipo
    
    

In [ ]:
#def wrapper function stepsize (adaptive step size driver for rk4), to carry out the function of the rkmv core
#^adapting size of step h based on error/tolerance of step from rk estimate of error in y&z

def rk4_mv_ad(dydx, x_i, y_i, nv, h, tol):
    
    #def safety scale
    SAFETY = 0.9
    H_NEW_FAC = 2.0
    
    #set max # iterations
    imax = 10000
    
    #set iterator
    i = 0
    
    #error array
    Delta = np.full(nv, 2*tol) #(while loop) starts at error above tolerance
    
    #remember step
    h_step = h
    
    #adjust step
    while(Delta.max()/tol>1.0):
        #est error by taking one step of size h & 2 of size h/2 (in order), based on grid of y calc from rk_mv_core
        #^2 different steps est to make sure error is not too big
        #smaller intervals = better error, esp for rk4
        y_2 = rk_mv_core(dydx, x_i, y_i, nv, h_step)
        y_1 = rk_mv_core(dydx, x_i, y_i, nv, 0.5*h_step)
        y_11 = rk_mv_core(dydx, x_i+0.5*h_step, y_1, nv, 0.5*h_step)
        
        #compare errors
        Delta = np.fabs(y_2 - y_11)
        
        #if error too large, take smaller step
        if(Delta.max()/tol>1.0):
            h_step*= SAFETY*(Delta.max()/tol)**(-0.25)
            
        #check iteration
        if(i>=imax):
            print("too many iterations in rk4_mv_ad")
            raise StopIteration("ending after i=",i)
        
        #iterate
        i +=1
        
    #outside while loop, get next (new) step
    h_new = np.fmin(h_step*(Delta.max()/tol)**(-0.9), h_step*H_NEW_FAC)
    
    #return answer, a new step, and the step we took (finite different steps)
    return y_2, h_new, h_step

    

In [ ]:
#def wrapper for rk4 mv core
def rk4_mv(dfdx, a, b, y_a, tol):
    #def starting step
    xi = a
    yi = y_a.copy() #copy y array from before
    
    #initial step
    h = 1.0e-4 *(b-a)
    
    #set max number of iterations
    imax = 10000
    
    #set iteration variable
    i = 0
    
    #set number coupled odes as nv (variables z&y at the same time - via the use of y_a - copy of previous 2-element yi array)
    nv = len(y_a)
    
    #set initial conditions
    x = np.full(1,a)
    y = np.full((1,nv), y_a)
    
    #set flag
    flag = 1
    #loop until we reach the right side
    while(flag):
        #calc y_ipo
        yi_new, h_new, h_step = rk4_mv_ad(dydx, xi, yi, nv, h, tol)
        #update step, based on adaptive stepsize driver rk4_mv_ad
        h = h_new
        
        #prevent an overshoot
        if(xi+h_step>b):
            #if xi+step greater than upper limit, take a smaller step
            h = b-xi
            
            #recalculate y_ipo
            yi_new, h_new, h_step = rk4_mv_ad(dydx, xi, yi, nv, h, tol)
            
            #break from while loop
            flag = 0
        
        #update values
        xi += h_step
        
        yi[:] = yi_new[:]
        
        #add step to the arrays
        x = np.append(x, xi)
        #append yi to the end, delete old y, save ynew into y
        y_new = np.zeros((len(x), nv))
        y_new[0:len(x)-1, :] = y
        y_new[-1,:] = yi[:]
        del y
        
        y = y_new
        
        #prevent too many iterations
        if(i>=imax):
            print("max iterations reached")
            raise StopIteration("iteration # =",i)
            
        #iterate
        i += 1
        
        #print out some info
        s = "i = %3d\tx = %9.8f\th = %9.8f\tb = %9.8f" % (i, xi, h_step, b)
        print(s)
        
        #break if xi = b
        if (xi==b):
            flag = 0
    #outside while loop, return the answer (evolved x&y values)
    return x,y

In [ ]:
#perform the rk integration for the 2 coupled derivatives

#bounds (0, 2pi)
a = 0.0
b = 2.0*np.pi

#y array
y_0 = np.zeros(2)
y_0[0] = 0.0 #initially, y(x=0) = 0 = y[0]
y_0[1] = 1.0 #initially, dydx(x=0) = 1 = z = y[1]

#variables to integrate, z&y
nv = 2
tolerance = 1.0e-6

#perform integration analytically
x,y = rk4_mv(dydx, a, b, y_0, tolerance)

In [ ]:
#plot analytical result of x,y against sinx&cosx
plt.plot(x,y[:,0], 'o', label='y(x)')
plt.plot(x,y[:,1], 'o', label='dydx(x)')
xx = np.linspace(0,2.0*np.pi,1000)
plt.plot(xx, np.sin(xx), label='sin(x)')
plt.plot(xx, np.cos(xx), label='cos(x)')
plt.xlabel('x')
plt.ylabel('y, dy/dx')
plt.legend(frameon=False)

In [ ]:
#plot the error
sine = np.sin(x)
cosine = np.cos(x)

y_error = (y[:,0]-sine)
dydx_error = (y[:,1]-cosine)

plt.plot(x, y_error, label="y(x) error")
plt.plot(x, dydx_error, label="dydx(x) error")
plt.legend(frameon=False)